## MSM.jl and Surrogates.jl

This notebook shows how one can estimate a model using [MSM.jl](https://github.com/JulienPascal/MSM.jl) and [Surrogates.jl](https://github.com/SciML/Surrogates.jl)

In [ ]:
using MSM
using Surrogates
using DataStructures
using OrderedCollections
using Distributions
using Random
using DataStructures
using Statistics
using LinearAlgebra

In [ ]:
Random.seed!(1234)  #for replicability reasons
T = 100000          #number of periods
P = 2               #number of dependent variables
beta0 = rand(P)     #choose true coefficients by drawing from a uniform distribution on [0,1]
alpha0 = rand(1)[]  #intercept
theta0 = 0.0        #coefficient to create serial correlation in the error terms
println("True intercept = $(alpha0)")
println("True coefficient beta0 = $(beta0)")
println("Serial correlation coefficient theta0 = $(theta0)")

# Generation of error terms
# row = individual dimension
# column = time dimension 
U = zeros(T)
d = Normal()
U[1] = rand(d, 1)[] #first error term
# loop over time periods
for t = 2:T
    U[t] = rand(d, 1)[] + theta0*U[t-1]
end

# Let's simulate the dependent variables x_t
x = zeros(T, P)

d = Uniform(0, 5)
for p = 1:P  
    x[:,p] = rand(d, T)
end

# Let's calculate the resulting y_t
y = zeros(T)

for t=1:T
    y[t] = alpha0 + x[t,1]*beta0[1] + x[t,2]*beta0[2] + U[t]
end

optionsSMM = MSMOptions(maxFuncEvals=1000, globalOptimizer = :dxnes, localOptimizer = :NelderMead)
myProblem = MSMProblem(options = optionsSMM);

# Priors
dictPriors = OrderedDict{String,Array{Float64,1}}()
dictPriors["alpha"] = [0.5, 0.001, 1.0]
dictPriors["beta1"] = [0.5, 0.001, 1.0]
dictPriors["beta2"] = [0.5, 0.001, 1.0]
set_priors!(myProblem, dictPriors)

# Empirical moments
dictEmpiricalMoments = OrderedDict{String,Array{Float64,1}}()
dictEmpiricalMoments["mean"] = [mean(y); mean(y)] #informative on the intercept
dictEmpiricalMoments["mean_x1y"] = [mean(x[:,1] .* y); mean(x[:,1] .* y)] #informative on betas
dictEmpiricalMoments["mean_x2y"] = [mean(x[:,2] .* y); mean(x[:,2] .* y)] #informative on betas
dictEmpiricalMoments["mean_x1y^2"] = [mean((x[:,1] .* y).^2); mean((x[:,1] .* y).^2)] #informative on betas
dictEmpiricalMoments["mean_x2y^2"] = [mean((x[:,2] .* y).^2); mean((x[:,2] .* y).^2)] #informative on betas
set_empirical_moments!(myProblem, dictEmpiricalMoments)

W = Matrix(1.0 .* I(length(dictEmpiricalMoments)))#initialization
#Special case: diagonal matrix
#(you may choose something else)
for (indexMoment, k) in enumerate(keys(dictEmpiricalMoments))
    W[indexMoment,indexMoment] = 1.0/(dictEmpiricalMoments[k][1])^2
end
set_weight_matrix!(myProblem, W)

# x[1] corresponds to the intercept, x[2] corresponds to beta1, x[3] corresponds to beta2
function functionLinearModel(x; uniform_draws::Array{Float64,1}, simX::Array{Float64,2}, nbDraws::Int64 = length(uniform_draws), burnInPerc::Int64 = 0)
    T = nbDraws
    P = 2       #number of dependent variables

    alpha = x[1]
    beta = x[2:end]
    theta = 0.0     #coefficient to create serial correlation in the error terms

    # Creation of error terms
    # row = individual dimension
    # column = time dimension
    U = zeros(T)
    d = Normal()
    # Inverse cdf (i.e. quantile)
    gaussian_draws = quantile.(d, uniform_draws)
    U[1] = gaussian_draws[1] #first error term

    # loop over time periods
    for t = 2:T
        U[t] = gaussian_draws[t] + theta*U[t-1]
    end

    # Let's calculate the resulting y_t
    y = zeros(T)

    for t=1:T
        y[t] = alpha + simX[t,1]*beta[1] + simX[t,2]*beta[2] + U[t]
    end

    # Get rid of the burn-in phase:
    #------------------------------
    startT = max(1, Int(nbDraws * (burnInPerc / 100)))

    # Moments:
    #---------
    output = OrderedDict{String,Float64}()
    output["mean"] = mean(y[startT:nbDraws])
    output["mean_x1y"] = mean(simX[startT:nbDraws,1] .* y[startT:nbDraws])
    output["mean_x2y"] = mean(simX[startT:nbDraws,2] .* y[startT:nbDraws])
    output["mean_x1y^2"] = mean((simX[startT:nbDraws,1] .* y[startT:nbDraws]).^2)
    output["mean_x2y^2"] = mean((simX[startT:nbDraws,2] .* y[startT:nbDraws]).^2)

    return output
end

# Let's freeze the randomness during the minimization
d_Uni = Uniform(0,1)
nbDraws = T #number of draws in the simulated data
uniform_draws = rand(d_Uni, nbDraws)
simX = zeros(length(uniform_draws), 2)
d = Uniform(0, 5)
for p = 1:2
  simX[:,p] = rand(d, length(uniform_draws))
end

set_simulate_empirical_moments!(myProblem, x -> functionLinearModel(x, uniform_draws = uniform_draws, simX = simX))
construct_objective_function!(myProblem)

In [ ]:
myProblem.objective_function([dictPriors[k][1] for k in keys(dictPriors)])

### Sampling

In [ ]:
n_samples = 50
lower_bound = create_lower_bound(myProblem)
upper_bound = create_upper_bound(myProblem)
xys = Surrogates.sample(n_samples, lower_bound, upper_bound, SobolSample());
size(xys)

In [ ]:
g(x) = myProblem.objective_function(x)
zs = g.(xys);
size(zs)

In [ ]:
xs = [xy[1] for xy in xys]
ys = [xy[2] for xy in xys]
zs = [xy[3] for xy in xys]
scatter(xs, ys, zs, zcolor=zs)

### Building a surrogate

In [ ]:
kriging_surrogate = Kriging(xys, zs, lower_bound, upper_bound, p=collect([1.5 for k in 1:length(dictPriors)]));

### Optimizing

In [ ]:
@time min_surrogate = surrogate_optimize(g, SRBF(), lower_bound, upper_bound, kriging_surrogate, SobolSample(), maxiters=100)

In [ ]:
size(xys)

In [ ]:
println("Minimum objective function = $(min_surrogate[2]) \n")
println("Estimated value for alpha = $(min_surrogate[1][1]). True value for beta1 = $(alpha0[1]) \n")
println("Estimated value for beta1 = $(min_surrogate[1][2]). True value for beta1 = $(beta0[1]) \n")
println("Estimated value for beta2 = $(min_surrogate[1][3]). True value for beta2 = $(beta0[2]) \n")

### Plotting the surrogate

Let's plot the objective function, **holding constant the value of $\alpha$**:

In [ ]:
using Plots
gr()

p1 = plot(collect(lower_bound[2]:0.1:upper_bound[2]), collect(lower_bound[3]:0.1:upper_bound[3]), (x, y) -> kriging_surrogate([alpha0 x y]), linetype=:surface)
xs = [xy[2] for xy in xys]
ys = [xy[3] for xy in xys]
gg(x) = g([alpha0; x[2]; x[3]])
zs = gg.(xys) 
scatter!(xs, ys, zs, marker_z=zs)
p2 = contour(collect(lower_bound[2]:0.1:upper_bound[2]), collect(lower_bound[3]:0.1:upper_bound[3]), (x, y) -> kriging_surrogate([alpha0 x y]))
scatter!(xs, ys, marker_z=zs) 
plot(p1, p2) 

Let's plot the objective function, **holding constant the value of $\beta_2$**:

In [ ]:
p1 = plot(collect(lower_bound[1]:0.1:upper_bound[1]), collect(lower_bound[2]:0.1:upper_bound[2]), (x, y) -> kriging_surrogate([x y beta0[2]]), linetype=:surface)
xs = [xy[1] for xy in xys]
ys = [xy[2] for xy in xys]
gg(x) = g([x[1]; x[2]; beta0[2]])
zs = gg.(xys) 
scatter!(xs, ys, zs, marker_z=zs)
p2 = contour(collect(lower_bound[1]:0.1:upper_bound[1]), collect(lower_bound[2]:0.1:upper_bound[2]), (x, y) -> kriging_surrogate([x y beta0[2]]))
scatter!(xs, ys, marker_z=zs) 
plot(p1, p2)

### Explore the sampling process

In [ ]:
using DataFrames
using StatsPlots
df = DataFrame(x1 = xs, x2 = ys)
@df df marginalhist(:x1, :x2)

In [ ]:
versioninfo()